In [11]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor, PILToTensor, Compose, Normalize
import torch.nn.functional as F
import time
import csv
torch.set_num_threads(1)


In [12]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [13]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
    # transform=PILToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [14]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

class CifarNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [15]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X.float())
        # print("pred: ",pred.shape)
        # print("y:",y.shape)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [16]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    return correct

In [17]:
transform = Compose(
    [
        ToTensor(),
        Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ]
)


# Download training data from open datasets.
training_data = datasets.CIFAR10(
    root="data",
    train=True,
    download=True,
    transform=transform,
    # transform=PILToTensor(),
)

# Download test data from open datasets.
test_data = datasets.CIFAR10(
    root="data",
    train=False,
    download=True,
    transform=transform,
)

Files already downloaded and verified
Files already downloaded and verified


In [18]:
loss_fn = nn.CrossEntropyLoss()
# loss_fn = nn.MSELoss()

In [19]:
test_dataloader = DataLoader(test_data, batch_size=64)


epochs = 3
for bsize in [4,8]:
    model = CifarNetwork().to(device)
    optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
    for epoch in range(epochs):

        train_dataloader_subset = DataLoader(training_data, batch_size=bsize)
        print(f"Epoch {epoch+1}\n-------------------------------")
        epoch_time_start = time.perf_counter()
        train(train_dataloader_subset, model, loss_fn, optimizer)
        epoch_time_end = time.perf_counter()

        test_start_time = epoch_time_end
        accuracy = test(test_dataloader, model, loss_fn)
        test_end_time = time.perf_counter()

        epoch_duration = epoch_time_end - epoch_time_start
        test_duration = test_end_time - test_start_time

        print(f"Epoch {epoch+1} took {epoch_duration} seconds")
        with open("minibatch_cifar_nn.csv", "a") as fp:
            wr = csv.writer(fp, dialect='excel')
            # epoch_duration, epoch, batch_size, data_size, accuracy, test_duration
            wr.writerow([epoch_duration, epoch, bsize, 50000, accuracy, test_duration])

print("Done!")

Epoch 1
-------------------------------
loss: 2.313524  [    4/50000]
loss: 2.270159  [  404/50000]
loss: 2.288055  [  804/50000]
loss: 2.293899  [ 1204/50000]
loss: 2.325410  [ 1604/50000]
loss: 2.304053  [ 2004/50000]
loss: 2.274299  [ 2404/50000]
loss: 2.324052  [ 2804/50000]
loss: 2.316264  [ 3204/50000]
loss: 2.344781  [ 3604/50000]
loss: 2.306680  [ 4004/50000]
loss: 2.276045  [ 4404/50000]
loss: 2.313386  [ 4804/50000]
loss: 2.334394  [ 5204/50000]
loss: 2.297205  [ 5604/50000]
loss: 2.309811  [ 6004/50000]
loss: 2.281672  [ 6404/50000]
loss: 2.273612  [ 6804/50000]
loss: 2.285515  [ 7204/50000]
loss: 2.324778  [ 7604/50000]
loss: 2.328305  [ 8004/50000]
loss: 2.309227  [ 8404/50000]
loss: 2.307688  [ 8804/50000]
loss: 2.265779  [ 9204/50000]
loss: 2.323225  [ 9604/50000]
loss: 2.300496  [10004/50000]
loss: 2.294869  [10404/50000]
loss: 2.311247  [10804/50000]
loss: 2.284938  [11204/50000]
loss: 2.321894  [11604/50000]
loss: 2.312114  [12004/50000]
loss: 2.263813  [12404/50000]


In [20]:
batch_size = 4
test_dataloader = DataLoader(test_data, batch_size=4)


epochs = 3
for datasize in [7500,15000,30000,50000]:
    model = CifarNetwork().to(device)
    optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

    for epoch in range(epochs):
        subset_training_data = Subset(training_data, range(datasize))
        train_dataloader_subset = DataLoader(subset_training_data, batch_size=batch_size)

        print(f"Epoch {epoch+1}\n-------------------------------")
        epoch_time_start = time.perf_counter()
        train(train_dataloader_subset, model, loss_fn, optimizer)
        epoch_time_end = time.perf_counter()

        test_start_time = epoch_time_end
        accuracy = test(test_dataloader, model, loss_fn)
        test_end_time = time.perf_counter()

        epoch_duration = epoch_time_end - epoch_time_start
        test_duration = test_end_time - test_start_time

        print(f"Epoch {epoch+1} took {epoch_duration} seconds")
        with open("datasize_cifar_nn.csv", "a") as fp:
            wr = csv.writer(fp, dialect='excel')
            # epoch_duration, epoch, batch_size, data_size, accuracy, test_duration
            wr.writerow([epoch_duration, epoch, batch_size, datasize, accuracy, test_duration])

print("Done!")

Epoch 1
-------------------------------
loss: 2.327474  [    4/ 7500]
loss: 2.261148  [  404/ 7500]
loss: 2.245978  [  804/ 7500]
loss: 2.303059  [ 1204/ 7500]
loss: 2.350530  [ 1604/ 7500]
loss: 2.333683  [ 2004/ 7500]
loss: 2.311251  [ 2404/ 7500]
loss: 2.334387  [ 2804/ 7500]
loss: 2.294865  [ 3204/ 7500]
loss: 2.355238  [ 3604/ 7500]
loss: 2.301446  [ 4004/ 7500]
loss: 2.268570  [ 4404/ 7500]
loss: 2.306024  [ 4804/ 7500]
loss: 2.344612  [ 5204/ 7500]
loss: 2.292885  [ 5604/ 7500]
loss: 2.306342  [ 6004/ 7500]
loss: 2.295149  [ 6404/ 7500]
loss: 2.261823  [ 6804/ 7500]
loss: 2.296309  [ 7204/ 7500]
Test Error: 
 Accuracy: 10.0%, Avg loss: 2.302746 

Epoch 1 took 3.8937714600469917 seconds
Epoch 2
-------------------------------
loss: 2.312066  [    4/ 7500]
loss: 2.269509  [  404/ 7500]
loss: 2.260037  [  804/ 7500]
loss: 2.300740  [ 1204/ 7500]
loss: 2.340050  [ 1604/ 7500]
loss: 2.317119  [ 2004/ 7500]
loss: 2.307583  [ 2404/ 7500]
loss: 2.323736  [ 2804/ 7500]
loss: 2.290844  [ 

IndexError: index 50000 is out of bounds for axis 0 with size 50000

In [ ]:
torch.save(model.state_dict(), "cifar_model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [ ]:
model = CifarNetwork().to(device)
model.load_state_dict(torch.load("cifar_model.pth"))

RuntimeError: Error(s) in loading state_dict for NeuralNetwork:
	Missing key(s) in state_dict: "linear_relu_stack.0.weight", "linear_relu_stack.0.bias", "linear_relu_stack.2.weight", "linear_relu_stack.2.bias", "linear_relu_stack.4.weight", "linear_relu_stack.4.bias". 
	Unexpected key(s) in state_dict: "conv1.weight", "conv1.bias", "conv2.weight", "conv2.bias", "fc1.weight", "fc1.bias", "fc2.weight", "fc2.bias", "fc3.weight", "fc3.bias". 